# Data Preprocessor
Dieses Notebook bereitet die Daten aus **1_sources** auf und speichert sie in einem einheitlichen CSV-Format im Ordner **2_preprocessed**.
Die einzelnen Datensätze wurden überprüft, gefundene Fehler werden in diesem Notebook behoben. Am Ende des Notebooks werden die aufbereiteten Daten validiert.

In [1]:
import pandas as pd
import os
from config import SOURCE_BFS_DATA_FOREIGNERS, SOURCE_STADA_SOCIAL_ASSISTANCE_RATE, SOURCE_STADA_FISCAL_POWER, SOURCE_WIKIPEDIA_COMMUNITY_STATS, PREPROCESSED_DIR, PREPROCESSED_BFS_DATA_FOREIGNERS, PREPROCESSED_STADA_SOCIAL_ASSISTANCE_RATE, PREPROCESSED_STADA_FISCAL_POWER, PREPROCESSED_WIKIPEDIA_COMMUNITY_STATS

os.makedirs(PREPROCESSED_DIR, exist_ok=True)

## Source: Statistikdatenbank St. Gallen (Stada)
Für diesen Datensatz musten folgende Korrekturen vorgenommen werden:
* Korrektur Gemeindenamen: Gemeinden, welche Anfangs 2023 fusioniert haben, haben bei dieser Datenquellen einen entsprechenden Zusatz im Namen "(2022)". Dieser Zusatz wurde entfernt. Zudem musste die Schreibweise von St. Gallen und St. Margrethen denjenigen der anderen Quellen angeglichen werden
* Ergänzung fehlender Sozialhilfequote: Bei den Gemeinden, welche im 2023 fusionierten, waren keine Zahlen für die Sozialhilfequote im Jahr 2022 in den heruntergeladenen Daten vorhanden. Die Daten konnten aber beim Statistikamt des Kantons St. Gallen in einem Bericht über die Gemeinden ausfindig gemacht werden.

In [2]:
def process_stada_data(input_file_name, output_file_name):

    data = pd.read_excel(input_file_name, names=['bfs_nr', 'community_name', 'category', 'dataset', 'subset', 'indicator_id', 'indicator_name', 'indicator_year', 'value', 'unit_short', 'unit_long'])
    
    # cleanup community names
    data['community_name'].replace({
        'Hemberg (bis 2022)': 'Hemberg',
        'Neckertal (bis 2022)': 'Neckertal',
        'Oberhelfenschwil (bis 2022)': 'Oberhelfenschwil',
        'Stadt St.Gallen': 'St. Gallen',
        'St.Margrethen': 'St. Margrethen'
    }, inplace=True)

    if(input_file_name == SOURCE_STADA_SOCIAL_ASSISTANCE_RATE):
        # social assistance rate is missing for communities which will merge in 2023. Data is available here: https://www.sg.ch/content/dam/sgch/kanton-stgallen/statistik/b13/B13_01_Gemeindekennzahlen.pdf
        data.loc[data['community_name'] == 'Neckertal', 'value'] = 2.0
        data.loc[data['community_name'] == 'Hemberg', 'value'] = 0.6
        data.loc[data['community_name'] == 'Oberhelfenschwil', 'value'] = 0.3
    
    # sort by community_name and write to csv
    columns_to_write = ['bfs_nr', 'community_name', 'value']
    data = data.sort_values(by='community_name').reset_index(drop=True)
    data[columns_to_write].to_csv(output_file_name, index=False)
    
    return data['bfs_nr'], data['community_name']

In [3]:
bfs_nrs_stada_fiscal_power, community_names_stada_fiscal_power = process_stada_data(SOURCE_STADA_FISCAL_POWER, PREPROCESSED_STADA_FISCAL_POWER)
bfs_nrs_stada_social_assistance, community_names_stada_social_assistance = process_stada_data(SOURCE_STADA_SOCIAL_ASSISTANCE_RATE, PREPROCESSED_STADA_SOCIAL_ASSISTANCE_RATE)

C:\Users\dario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\dario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Source: Bundesamt für Statistik (BFS)
Für diese Quelle mussten keine Daten korrigiert werden. Es wurden lediglich Spalten entfernt, welche nicht benötigt werden.

Das Datenset beinhaltet Werte für alle Gemeinden der Schweiz, weswegen eine Filterung auf die Gemeinden des Kantons St. Gallen vorgenommen werden muss.

In [4]:
def process_bfs_data(input_file_name, output_file_name):
    data = pd.read_excel(input_file_name, skiprows=6, skipfooter=11, header=None, names=['bfs_nr', 'community_name', 'number_of_foreigners', 'foreigners_percentage'])

    # filter communities in canton St. Gallen
    data = pd.merge(data, bfs_nrs_stada_fiscal_power, on='bfs_nr', how='inner')

    # sort by community_name and write to csv
    data = data.sort_values(by='community_name').reset_index(drop=True)
    data.to_csv(output_file_name, index=False)

    return data['bfs_nr'], data['community_name']

In [5]:
bfs_nrs_bfs, community_names_bfs = process_bfs_data(SOURCE_BFS_DATA_FOREIGNERS, PREPROCESSED_BFS_DATA_FOREIGNERS)

## Source: Wikipedia
Für diesen Datensatz musten folgende Korrekturen vorgenommen werden:
* Ersetzung von Hochkommas in Zahlen-Attributen
* Korrektur von Gemeindenamen: Rüthi musste der Schreibweise der anderen Datenquellen angepasst werden
* Einfügen von fehlenden Werten: Gemeinden, welche 2023 fusioniert haben, sind für das Jahr 2022 keine Daten vorhanden. In den einzelnen Wikipedia-Artikel konnten die Informationen jedoch gefunden werden.

In [6]:
def replace_values(data, community_name, values):
    matching_rows = data['community_name'] == community_name
    data.loc[matching_rows, ['number_of_inhabitants', 'area', 'inhabitants_per_km2']] = values

def process_wikipedia_data(input_file_name, output_file_name):
    columns_to_write = ['community_name', 'number_of_inhabitants', 'area', 'inhabitants_per_km2', 'district', 'electoral_district']
    data = pd.read_csv(input_file_name, engine='python', skiprows=1, skipfooter=1, names=['coat_of_arms', 'community_name', 'number_of_inhabitants', 'area', 'inhabitants_per_km2', 'district', 'electoral_district'])

    # cleanup community names
    data['community_name'].replace('Rüthi', 'Rüthi (SG)', inplace=True)

    # communities which merged in 2023 are not displayed correctly in the wikipedia-article-version in 2022.
    replace_values(data, "Hemberg", ['954', '20.15', '47'])           # source: https://de.wikipedia.org/w/index.php?title=Hemberg_SG&oldid=237114098
    replace_values(data, "Neckertal", ['6324', '81.84', '77'])        # source: https://de.wikipedia.org/w/index.php?title=Neckertal&oldid=241298594
    replace_values(data, "Oberhelfenschwil", ['1246', '12.68', '98']) # source: https://de.wikipedia.org/w/index.php?title=Oberhelfenschwil&oldid=242348540

    # fix numbers formats
    data['number_of_inhabitants'] = data['number_of_inhabitants'].str.replace('\'', '')
    data['area'] = data['area'].str.replace('\'', '')
    data['inhabitants_per_km2'] = data['inhabitants_per_km2'].str.replace('\'', '')

    # sort by community_name and write to csv
    data = data.sort_values(by='community_name').reset_index(drop=True)
    data[columns_to_write].to_csv(output_file_name, index=False)

    return data['community_name']

In [7]:
community_names_wikipedia = process_wikipedia_data(SOURCE_WIKIPEDIA_COMMUNITY_STATS, PREPROCESSED_WIKIPEDIA_COMMUNITY_STATS)

## Validierung der aufbereiteten Daten
In diesem Abschnitt werden die Datensätze gegeneinander validiert:
* Überprüfung, ob die BFS-Nummern in allen Datensätzen identisch sind (Wikipedia-Datensatz beinhaltet keine BFS-Nummern)
* Überprüfung, ob die Gemeinde-Namen in allen Datensätzen identisch sind
* Überprüfung, ob alle Attribute aller Datensätze einen Wert haben
* Überprüfung, ob numerische Attribute numerische Werte haben
* Überprüfung, ob Attribute mit prozentualen Werten ausschliesslich Werte zwischen 0 und 100 haben

In [8]:
assert bfs_nrs_stada_fiscal_power.equals(bfs_nrs_stada_social_assistance)
assert bfs_nrs_stada_social_assistance.equals(bfs_nrs_bfs)

assert community_names_stada_fiscal_power.equals(community_names_stada_social_assistance) 
assert community_names_stada_social_assistance.equals(community_names_bfs)
assert community_names_bfs.equals(community_names_wikipedia)

bfs_foreigners = pd.read_csv(PREPROCESSED_BFS_DATA_FOREIGNERS)
stada_fiscal_power = pd.read_csv(PREPROCESSED_STADA_FISCAL_POWER)
stada_social_assistance = pd.read_csv(PREPROCESSED_STADA_SOCIAL_ASSISTANCE_RATE)
wikipedia_data = pd.read_csv(PREPROCESSED_WIKIPEDIA_COMMUNITY_STATS)

assert pd.to_numeric(bfs_foreigners['bfs_nr'], errors='coerce').notna().all()
assert bfs_foreigners['community_name'].notna().all()
assert pd.to_numeric(bfs_foreigners['number_of_foreigners'], errors='coerce').notna().all()
assert pd.to_numeric(bfs_foreigners['foreigners_percentage'], errors='coerce').notna().all()
assert ((bfs_foreigners['foreigners_percentage'] >= 0) & (bfs_foreigners['foreigners_percentage'] <= 100)).all()

assert pd.to_numeric(stada_fiscal_power['bfs_nr'], errors='coerce').notna().all()
assert stada_fiscal_power['community_name'].notna().all()
assert pd.to_numeric(stada_fiscal_power['value'], errors='coerce').notna().all()

assert pd.to_numeric(stada_social_assistance['bfs_nr'], errors='coerce').notna().all()
assert stada_social_assistance['community_name'].notna().all()
assert pd.to_numeric(stada_social_assistance['value'], errors='coerce').notna().all()
assert ((stada_social_assistance['value'] >= 0) & (stada_social_assistance['value'] <= 100)).all()

assert wikipedia_data['community_name'].notna().all()
assert pd.to_numeric(wikipedia_data['number_of_inhabitants'], errors='coerce').notna().all()
assert pd.to_numeric(wikipedia_data['area'], errors='coerce').notna().all()
assert pd.to_numeric(wikipedia_data['inhabitants_per_km2'], errors='coerce').notna().all()
assert wikipedia_data['district'].notna().all()
assert wikipedia_data['electoral_district'].notna().all()